In [10]:
from time import time
import pandas as pd
import numpy as np
from datetime import datetime
import scipy as sc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from binance import Client
import time

client = Client()


smooth_interval = 25
smooth_exp = 3
rango = 0
std_mult = 25
z_aprox = 0.15

start = time.time()
recent_data = pd.read_csv('../../../../../csvs/historical/by1m/BTCUSDT')
recent_data = recent_data.iloc[:1000,:6]
recent_data.columns = ['Tiempo','Open','High','Low','Close','Volume']
recent_data['Tiempo'] = recent_data['Tiempo'].astype('datetime64[ns]')# pd.to_datetime(recent_data.Tiempo,unit='ms')
recent_data = recent_data.set_index('Tiempo')
recent_data = recent_data.astype(float)
recent_data = recent_data.reset_index()
end = time.time()
print(f'{end-start}')
recent_data

2.1500744819641113


,Tiempo,Open,High,Low,Close,Volume
0,2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183
1,2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000
2,2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074
3,2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008
4,2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796
...,...,...,...,...,...,...
995,2017-08-17 20:35:00,4302.96,4302.96,4302.96,4302.96,0.052740
996,2017-08-17 20:36:00,4302.96,4302.96,4302.96,4302.96,0.585532
997,2017-08-17 20:37:00,4301.91,4301.91,4301.91,4301.91,0.445848
998,2017-08-17 20:38:00,4301.91,4301.91,4301.91,4301.91,0.372403


In [12]:
start = time.time()
recent_data['Interpolacion'] = sc.signal.savgol_filter(recent_data.Open,smooth_interval,smooth_exp)
recent_data['Primer_Derivada'] = recent_data.Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
recent_data['PD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Primer_Derivada,smooth_interval,smooth_exp)
recent_data['Segunda_Derivada'] = recent_data.PD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
recent_data['SD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Segunda_Derivada,smooth_interval,smooth_exp)
recent_data['Tercer_Derivada'] = recent_data.SD_Interpolacion.diff()/(recent_data.Tiempo.diff()/np.timedelta64(1, 's'))
recent_data['TD_Interpolacion'] = sc.signal.savgol_filter(recent_data.Tercer_Derivada,smooth_interval,smooth_exp)
end = time.time()
print("DERIVATIVES = %s" % (end - start))

DERIVATIVES = 0.0072023868560791016


In [13]:
def savgol_filter(y, window_size, poly_order):
    half_window = window_size // 2
    order_range = np.arange(poly_order+1)
    weight = np.zeros((window_size, poly_order+1))
    
    for i in range(-half_window, half_window+1):
        weight[i+half_window, :] = [i**j for j in order_range]
    
    weights = np.linalg.pinv(weight).T.sum(axis=0)
    
    smoothed = np.convolve(weights, y, mode='valid')
    
    return smoothed #smoothed_padded

def calculate_derivatives(interpolacion, times, derivative):
    d_interpolacion = np.diff(interpolacion) / times[2+derivative:]
    return d_interpolacion

start = time.time()
open_prices = recent_data['Open'].values
times = (np.diff(recent_data.Tiempo)/np.timedelta64(1, 's')).astype(float)

interpolacion = savgol_filter(open_prices, smooth_interval, 3)
primer_derivada = calculate_derivatives(interpolacion, times, 1)
pd_interpolacion = savgol_filter(primer_derivada, smooth_interval, 3)
segunda_derivada = calculate_derivatives(pd_interpolacion, times, 5)
sd_interpolacion = savgol_filter(segunda_derivada, smooth_interval, 3)
tercer_derivada = calculate_derivatives(sd_interpolacion, times, 9)
td_interpolacion = savgol_filter(tercer_derivada, smooth_interval, 3)
end = time.time()
print("CPU ON DERIVATIVES = %s" % (end - start))

CPU ON DERIVATIVES = 0.0029768943786621094


In [14]:
start = time.time()
min_max_sd = np.diff(recent_data.Open) / (np.diff(recent_data.Tiempo)/np.timedelta64(1, 's')).astype(float)
lista_min_max = recent_data[1:].iloc[(min_max_sd >= -z_aprox) & (min_max_sd <= z_aprox)]
indices_min_max = lista_min_max.index.tolist()

max_points_idx = []
min_points_idx = []
pmin_idx = 0
pmax_idx = 0

for i in range(len(indices_min_max)):
    PMSD = indices_min_max[i]
    if recent_data['SD_Interpolacion'][PMSD] > 0:
        pmax = recent_data['SD_Interpolacion'][PMSD]
        for i in range(rango+1):
            if (recent_data['SD_Interpolacion'][PMSD-i] > pmax) & (abs(recent_data['SD_Interpolacion'][PMSD-i]*100) > abs(np.std(recent_data['SD_Interpolacion'])*std_mult)):
                pmax_idx = PMSD-i
            elif (recent_data['SD_Interpolacion'][PMSD+i] > pmax) & (abs(recent_data['SD_Interpolacion'][PMSD+i]*100) > abs(np.std(recent_data['SD_Interpolacion'])*std_mult)):
                pmax_idx = PMSD+i
        max_points_idx.append(pmax_idx)
    elif recent_data['SD_Interpolacion'][PMSD] < 0:
        pmin = recent_data['SD_Interpolacion'][PMSD]
        for i in range(1,rango+1):
            if (recent_data['SD_Interpolacion'][PMSD-i] < pmin) & (abs(recent_data['SD_Interpolacion'][PMSD-i]*80) > abs(np.std(recent_data['SD_Interpolacion'])*std_mult)):
                pmin_idx = PMSD-i
            elif (recent_data['SD_Interpolacion'][PMSD+i] < pmin) & (abs(recent_data['SD_Interpolacion'][PMSD+i]*80) > abs(np.std(recent_data['SD_Interpolacion'])*std_mult)):
                pmin_idx = PMSD+i
        min_points_idx.append(pmin_idx)
end = time.time()
print(f'{end-start}')

# return max_points_idx, min_points_idx

0.04173088073730469


In [ ]:
start = time.time()
min_max_sd = np.diff(recent_data.Open) / (np.diff(recent_data.Tiempo)/np.timedelta64(1, 's')).astype(float)
lista_min_max = recent_data.iloc[(min_max_sd >= -z_aprox) & (min_max_sd <= z_aprox)]
indices_min_max = lista_min_max.index.tolist()

max_points_idx = []
min_points_idx = []
pmin_idx = 0
pmax_idx = 0


recent_data['sdi_updown'] = np.select(condlist=[(abs(recent_data['SD_Interpolacion']*100) > abs(np.std(recent_data['SD_Interpolacion'])*std_mult))],
                                        choicelist=[1],
                                        default=0)
recent_data['pdi_updown'] = np.select(condlist=[((recent_data.PD_Interpolacion[PMSD-i]*pdi_mult) > abs(np.std(recent_data.Open)*std_mult)),
                                                ((recent_data.PD_Interpolacion[PMSD-i]*pdi_mult) < -abs(np.std(recent_data.Open)*std_mult))],
                                                choicelist=[1,-1],
                                                default=0)
recent_data['sell_buy_side'] = np.select(condlist=[recent_data['SD_Interpolacion'][PMSD] > 0,recent_data['SD_Interpolacion'][PMSD] < 0],
                                        choicelist=[1,-1],
                                        default=0)


if (recent_data['sell_buy_side'] == 1) & (recent_data['sdi_updown'] == 1) & (recent_data['pdi_updown'] == 1):
        max_points_idx.append(pmax_idx)
elif (recent_data['sell_buy_side'] == -1) & (recent_data['sdi_updown'] == 1) & (recent_data['pdi_updown'] == -1):
        min_points_idx.append(pmin_idx)

end = time.time()
print(f'{end-start}')